In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras 
from keras import layers

In [56]:
Data = pd.read_csv('Spam Email raw text for NLP.csv',)
Data.head(10)

,CATEGORY,MESSAGE,FILE_NAME
0,1,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",00249.5f45607c1bffe89f60ba1ec9f878039a
1,1,ATTENTION: This is a MUST for ALL Computer Use...,00373.ebe8670ac56b04125c25100a36ab0510
2,1,This is a multi-part message in MIME format.\n...,00214.1367039e50dc6b7adb0f2aa8aba83216
3,1,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...,00210.050ffd105bd4e006771ee63cabc59978
4,1,This is the bottom line. If you can GIVE AWAY...,00033.9babb58d9298daa2963d4f514193d7d6
5,1,------=_NextPart_000_00B8_51E06B6A.C8586B31\n\...,00187.efd97ab2034b3384606e21db00014ecb
6,1,"<STYLE type=""text/css"">\n\n<!--\n\nP{\n\n fon...",00500.85b72f09f6778a085dc8b6821965a76f
7,1,<HR>\n\n<html>\n\n<head>\n\n <title>Secured I...,00493.1c5f59825f7a246187c137614fb1ea82
8,1,"<table width=""600"" border=""20"" align=""center"" ...",00012.381e4f512915109ba1e0853a7a8407b2
9,1,"<html>\n\n\n\n<head>\n\n<meta http-equiv=""Cont...",00487.139a2f4e8edbbdd64441536308169d74


In [57]:
Data.tail()

,CATEGORY,MESSAGE,FILE_NAME
5791,0,"I'm one of the 30,000 but it's not working ver...",00609.dd49926ce94a1ea328cce9b62825bc97
5792,0,Damien Morton quoted:\n\n>W3C approves HTML 4 ...,00957.e0b56b117f3ec5f85e432a9d2a47801f
5793,0,"On Mon, 2002-07-22 at 06:50, che wrote:\n\n\n\...",01127.841233b48eceb74a825417d8d918abf8
5794,0,"Once upon a time, Manfred wrote :\n\n\n\n> I w...",01178.5c977dff972cd6eef64d4173b90307f0
5795,0,"If you run Pick, and then use the ""New FTOC"" b...",00747.352d424267d36975a7b40b85ffd0885e


In [58]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5796 entries, 0 to 5795
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CATEGORY   5796 non-null   int64 
 1   MESSAGE    5796 non-null   object
 2   FILE_NAME  5796 non-null   object
dtypes: int64(1), object(2)
memory usage: 136.0+ KB


In [59]:
Data.describe()

,CATEGORY
count,5796.000000
mean,0.327122
std,0.469203
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [60]:
#Now that we have imported the data we need to understand some things
#1 what is our dataset saying to us 
#2 what are the  problems of our data
#3 what are the things not needed in my dataset
#4 which algorithm should i use is it supervised or unsupervised
#5 if supervised which algorithm is the best to solve my problem

In [61]:
def missing_data(Data):
    missing_data_count = Data.isnull().sum() * 100 / Data.shape[0]
    Data_missing_count = pd.DataFrame(missing_data_count).round(2)
    Data_missing_count = Data_missing_count.reset_index().rename(
                columns= {
                     'index': 'Column',
                      0 : 'Missing_Percentage',
                }
)
    Data_missing_value = Data.isnull().sum()
    Data_missing_value = Data_missing_value.reset_index().rename(
                columns= {
                    'index': 'Column',
                     0: 'Missing_Count',
                }
)
    Final = Data_missing_value.merge(Data_missing_count,how = 'inner', left_on = 'Column', right_on = 'Column')  
    Final = Final.sort_values(by = 'Missing_Count', ascending = False)
    return Final

missing_data(Data)
    

,Column,Missing_Count,Missing_Percentage
0,CATEGORY,0,0.0
1,MESSAGE,0,0.0
2,FILE_NAME,0,0.0


In [62]:
Data['MESSAGE'][0]

'Dear Homeowner,\n\n \n\nInterest Rates are at their lowest point in 40 years!\n\n\n\nWe help you find the best rate for your situation by\n\nmatching your needs with hundreds of lenders!\n\n\n\nHome Improvement, Refinance, Second Mortgage,\n\nHome Equity Loans, and More! Even with less than\n\nperfect credit!\n\n\n\nThis service is 100% FREE to home owners and new\n\nhome buyers without any obligation. \n\n\n\nJust fill out a quick, simple form and jump-start\n\nyour future plans today!\n\n\n\n\n\nVisit http://61.145.116.186/user0201/index.asp?Afft=QM10\n\n\n\n\n\n\n\n\n\n\n\n\n\nTo unsubscribe, please visit:\n\n\n\nhttp://61.145.116.186/light/watch.asp\n\n\n\n\n'

In [63]:
Data_shuffled = Data.sample(frac= 1, random_state = 42)

In [64]:
Data_shuffled.head(10)

,CATEGORY,MESSAGE,FILE_NAME
4675,0,"You could try HTTrack, available here: http://...",00420.f6140b71df992b02cc59548039eb05ca
4220,0,"At 01:12 AM 8/24/02 -0700, Adam L. Beberg wrot...",00314.611159749e214b996589d557e335648e
2484,0,Gary Lawrence Murphy wrote:\n\n>and say hello ...,00714.16c4d34ab2c9622fe82de9570946f9ef
2418,0,\n\n> (and no it wasnt me even though the spel...,00571.8f35c46bee6d7a238eabf207a5696b0c
5110,0,Gary Murphy:\n\n>How do you intend to take leg...,00991.80ac3c798bb00027efa6d80fdfb00d8e
1161,1,This is a multi-part message in MIME format.\n...,00497.353a61b265f11dd0bae116c0149abbe1
3296,0,URL: http://boingboing.net/#85531549\n\nDate: ...,02324.939312dace3fd8b6b3d2d1116cd83024
101,1,"\n\nHallo,\n\n\n\n\n\nI found yours Email ID i...",00360.3c1e6c84cb93d024c1f1aa85cd56ac9c
3337,0,>>\n\n>> http://www.post-gazette.com/columnist...,00811.caf6de89b99d2266288dfeff16359be3
401,1,FREE 30 day supply of HGH 1000:\n\nLook Younge...,00160.cec5f611ae665ff0add6c4928d47f2be


In [65]:
Data.CATEGORY.value_counts()

0    3900
1    1896
Name: CATEGORY, dtype: int64

In [92]:
import random
random_index = random.randint(0, len(Data) -5)
for row in Data_shuffled[['MESSAGE','CATEGORY']][random_index:random_index + 5].itertuples():
    __,MESSAGE,CATEGORY = row
    print(f'Target:{CATEGORY}','Ham' if CATEGORY < 1 else 'Spam')
    print(f'Text:\n{MESSAGE}\n')
    print('---\n')


Target:1 Spam
Text:
<html>

<head>

<title>index.gif</title>

<meta http-equiv="Content-Type" content="text/html;">

</head>

<body bgcolor="#006699" link="#FFFFFF">

<table border="0" cellpadding="0" cellspacing="0" width="542" align="center">

  <tr>

   <td><img src="http://209.126.159.62/spacer.gif" width="149" height="1" border="0"></td>

   <td><img src="http://209.126.159.62/spacer.gif" width="165" height="1" border="0"></td>

   <td><img src="http://209.126.159.62/spacer.gif" width="228" height="1" border="0"></td>

   <td><img src="http://209.126.159.62/spacer.gif" width="1" height="1" border="0"></td>

  </tr>



  <tr>

   <td><img name="index_r1_c1" src="http://209.126.159.62/index_r1_c1.gif" width="149" height="193" border="0"></td>

   <td><img name="index_r1_c2" src="http://209.126.159.62/index_r1_c2.gif" width="165" height="193" border="0"></td>

    <td rowspan="2"><a href="http://click.mm53.com/sp/t.pl?id=3217:7552100"><img name="index_r1_c3" src="http://209.126.159.6

In [67]:
Train_sentences,Val_sentences,Train_labels,Val_labels = train_test_split(Data_shuffled['MESSAGE'].to_numpy(),
                                                                         Data_shuffled['CATEGORY'].to_numpy(),
                                                                         test_size = 0.1,
                                                                         random_state=42)

In [68]:
len(Train_sentences),len(Train_labels),len(Val_sentences),len(Val_labels)

(5216, 5216, 580, 580)

In [69]:
len(Data_shuffled)

5796

In [70]:
Train_sentences[:1],Train_labels[:1]

(array(['I successfully installed spamassassin & razor to run system wide on my\n\nDebian Woody server.  \n\n \n\nBriefly I \n\napt-get installed spamassassin razor and libmilter-dev,\n\n \n\ndownloaded spamass-milter-0.1.2.tar.gz from http://www.milter.org,\n\nungzipped and untarred the file into /etc/mail, \n\n \n\nfollowed the directions in /etc/mail/spamass-milter-0.1.2/README to\n\ncompile the milter, install the rc scripts, and edit and update\n\nsendmail.mc\n\n \n\nchanged /etc/default/spamassassin to set spamassassin to daemon mode.\n\n \n\nverified that spamassassin was running by tailing /var/log/mail.log\n\n \n\n \n\nWoody/stable has SA 2.20\n\nWoody/unstable has SA 2.41\n\n\n\nI\'m running the stable source live right now and it is working very\n\nwell.\n\n\n\nIf you want unstable change /etc/apt/sources.list, substituting\n\n"unstable" for "stable",\n\nRun apt-get update\n\nInstall the unstable versions\n\nChange /etc/apt/sources.list\n\nRun apt-get update\n\n\n\n\n\nQuent

In [71]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [72]:
max_vocab_length = 1000
max_length = 15

In [73]:
Text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)

In [74]:
len(Train_sentences[0].split())

215

In [75]:
Train_sentences[0].split()

['I',
 'successfully',
 'installed',
 'spamassassin',
 '&',
 'razor',
 'to',
 'run',
 'system',
 'wide',
 'on',
 'my',
 'Debian',
 'Woody',
 'server.',
 'Briefly',
 'I',
 'apt-get',
 'installed',
 'spamassassin',
 'razor',
 'and',
 'libmilter-dev,',
 'downloaded',
 'spamass-milter-0.1.2.tar.gz',
 'from',
 'http://www.milter.org,',
 'ungzipped',
 'and',
 'untarred',
 'the',
 'file',
 'into',
 '/etc/mail,',
 'followed',
 'the',
 'directions',
 'in',
 '/etc/mail/spamass-milter-0.1.2/README',
 'to',
 'compile',
 'the',
 'milter,',
 'install',
 'the',
 'rc',
 'scripts,',
 'and',
 'edit',
 'and',
 'update',
 'sendmail.mc',
 'changed',
 '/etc/default/spamassassin',
 'to',
 'set',
 'spamassassin',
 'to',
 'daemon',
 'mode.',
 'verified',
 'that',
 'spamassassin',
 'was',
 'running',
 'by',
 'tailing',
 '/var/log/mail.log',
 'Woody/stable',
 'has',
 'SA',
 '2.20',
 'Woody/unstable',
 'has',
 'SA',
 '2.41',
 "I'm",
 'running',
 'the',
 'stable',
 'source',
 'live',
 'right',
 'now',
 'and',
 'it

In [76]:
round(sum([len(i.split()) for i in Train_sentences])/len(Train_sentences))

303

In [77]:
Text_vectorizer.adapt(Train_sentences)

In [78]:
sample_sentence = 'I love zendaya always and forever'
Text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 12, 767,   1, 391,   5,   1,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [79]:
# Get the unique words in the vocabulary
words_in_vocab = Text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 1000
Top 5 most common words: ['', '[UNK]', 'the', 'to', 'a']
Bottom 5 least common words: ['across', '00', 'various', 'told', 'thinking']


In [80]:
random_sentence = random.choice(Train_sentences)
print(f'Original text:\n {random_sentence}\
    \n\nVectorized version:')
Text_vectorizer([random_sentence])

Original text:
 URL: http://boingboing.net/#85516609

Date: Not supplied



Here's a brief interview with me.  



    *Interviewer:* If the Boing Boing zine and blog were TV characters, who'd 

    they be, and how'd they get on? 



    *Mark:* The zine would be Jethro Bodine of "The Beverly Hillbillies" -- 

    curious, neophilic, xenophilic, gleeful, and eager to adopt any new theory 

    or conspiracy as the absolute truth. The Blog would be Sherman from 

    "Peabody's Improbable History" -- a traveler of time and space in search of 

    beauty, truth, and the outre. I think Jethro bOING bOING and Sherman Boing 

    Boing would be great pals. Jethro would invite Sherman's dog, Mr. Peabody, 

    to go raccoon hunting with him, and Sherman would send Jethro 40 years into 

    the future to hang out at the Playboy Mansion.  



Link[1] Discuss[2]



[1] http://www.octapod.org/jeanpoole/archives/000203.html#000203

[2] http://www.quicktopic.com/16/H/kymC9LzuB2bHM




    

Vec

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[191,   1, 159,  25, 815,   1,   4,   1,   1,  18,  79,   1,  27,
          2,   1]], dtype=int64)>

In [81]:
from tensorflow.keras import  layers
embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 128,
                             embeddings_initializer='uniform',
                             input_length = max_length)
embedding

In [82]:
random_sentence = random.choice(Train_sentences)
print(f'Original text:\n {random_sentence}\
    \n\nEmbedded version:')
sample_embed = embedding(Text_vectorizer([random_sentence]))
sample_embed

Original text:
 <HTML>

<PRE>

Dear Valued Member,



OfferClub would like to thank you for the opportunity to present you

with valuable offers. You are on this list as you indicated that you

would like to receive special offers from trusted third parties via

email from us or one of our marketing partners. We maintain a high

level of customer satisfaction by ensuring periodically that we are

presenting third party information that you find important and useful.

To keep hearing from us every so often DO NOTHING.



We hope you'll stick around to check out our valuable content, but if

you do not wish to receive our emails please unsubscribe now by

clicking <a href="http://www.offerclubmail.com/unsubscribe.php?eid=\~moc.cnietonten^^mj\~1754388\~12a1">here</a>.





Thank you,



OfferClub 





--------------------------------------------------------------------------------

IMPORTANT: If this message was sent to you in error, or if you would

like to unsubscribe please click the 

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00641958,  0.02607188, -0.01266221, ...,  0.04758287,
         -0.02236451, -0.01996243],
        [ 0.00430795, -0.02319963, -0.02371956, ...,  0.02897375,
          0.03257183, -0.01661291],
        [ 0.0455935 , -0.00477745,  0.0062284 , ...,  0.03266852,
         -0.01154152, -0.03835007],
        ...,
        [-0.0366379 , -0.02374032,  0.00594819, ...,  0.02960796,
         -0.0497466 ,  0.01296153],
        [-0.02288141,  0.03784737,  0.04243047, ..., -0.02155271,
          0.03209014, -0.03363872],
        [-0.0038535 ,  0.0128735 ,  0.03752502, ...,  0.02933073,
          0.0354394 ,  0.00463455]]], dtype=float32)>

In [83]:
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.00641958,  0.02607188, -0.01266221,  0.01124877,  0.01936151,
       -0.00153504, -0.04222676,  0.03280507, -0.04943736,  0.03392353,
        0.00115705,  0.04404298, -0.0070901 , -0.0110525 ,  0.00393498,
        0.04765769, -0.03060706, -0.0317912 , -0.04191504, -0.01128843,
       -0.00700318, -0.01189746,  0.03849442,  0.00633273,  0.03244155,
        0.0068767 , -0.02379858, -0.00134418, -0.02923037, -0.02803289,
       -0.02833899, -0.02757443, -0.04924658,  0.03226452,  0.02363143,
       -0.02728986, -0.0447183 ,  0.04927154, -0.02446665, -0.02663103,
        0.02994666, -0.03094523, -0.03871783, -0.02487355, -0.04712589,
        0.01818434,  0.04863441, -0.0080476 , -0.02741168, -0.0183139 ,
        0.00015595,  0.04895555,  0.00454007, -0.04509979,  0.01016246,
       -0.02537006,  0.03110875, -0.04759628,  0.04175606,  0.01879804,
       -0.04269214,  0.03796183, -0.03726622, -0.01596794, -0.04029626,
        0.031503

Recurrent Neural Networks (RNN's), LSTM

In [84]:
from tensorflow.keras import layers
Inputs = layers.Input(shape = (1,), dtype = "string")
x = Text_vectorizer(Inputs)
x = embedding(x)
print(x.shape)
x = layers.LSTM(64,return_sequences=True)(x)
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
x = layers.Dense(64, activation = 'relu')(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model = tf.keras.Model(Inputs,outputs, name = 'model_LSTM')

(None, 15, 128)
(None, 15, 64)
(None, 64)


In [93]:
# Compile model
model.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [94]:
model__history = model.fit(Train_sentences,
                              Train_labels,
                              epochs=5,
                              validation_data=(Val_sentences, Val_labels),)

Epoch 1/5
163/163 [==============================] - 12s 36ms/step - loss: 0.2755 - accuracy: 0.8809 - val_loss: 0.1114 - val_accuracy: 0.9534
Epoch 2/5
163/163 [==============================] - 4s 27ms/step - loss: 0.1068 - accuracy: 0.9613 - val_loss: 0.1044 - val_accuracy: 0.9500
Epoch 3/5
163/163 [==============================] - 4s 28ms/step - loss: 0.0809 - accuracy: 0.9684 - val_loss: 0.1183 - val_accuracy: 0.9569
Epoch 4/5
163/163 [==============================] - 4s 27ms/step - loss: 0.0648 - accuracy: 0.9766 - val_loss: 0.1442 - val_accuracy: 0.9569
Epoch 5/5
163/163 [==============================] - 5s 30ms/step - loss: 0.0584 - accuracy: 0.9772 - val_loss: 0.1447 - val_accuracy: 0.9534


In [97]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [95]:
model_pred_probs = model.predict(Val_sentences)
model_pred_probs[:10]

19/19 [==============================] - 2s 22ms/step


array([[9.9908739e-01],
       [8.4197754e-01],
       [5.6300443e-05],
       [5.4105092e-03],
       [5.3314314e-05],
       [9.9913919e-01],
       [5.3419659e-05],
       [6.4762935e-05],
       [9.9902409e-01],
       [7.4449611e-01]], dtype=float32)

In [96]:
model_preds = tf.squeeze(tf.round(model_pred_probs))
model_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 0., 0., 0., 1., 0., 0., 1., 1.], dtype=float32)>

In [98]:
model_results = calculate_results(Val_labels, model_preds)
model_results

{'accuracy': 95.34482758620689,
 'precision': 0.9541102471406977,
 'recall': 0.9534482758620689,
 'f1': 0.9536296881397267}

In [99]:
val_df = pd.DataFrame({"text": Val_sentences,
                       "target": Val_labels,
                       "pred": model_preds,
                       "pred_prob": tf.squeeze(model_pred_probs)})
val_df.head()

,text,target,pred,pred_prob
0,<html>\n\n<head>\n\n</head>\n\n<body>\n\n<p al...,1,1.0,0.999087
1,¤£¦n·N«ä¥´ÅÍ±z¡I¦pªG¥»¼s§i«H¹ï±z²£¥Í§xÂZ¡A¦b¦¹...,1,1.0,0.841978
2,"On Mon, 23 Sep 2002, Gordon Mohr wrote:\n\n--]...",0,0.0,0.000056
3,"[Tim]\n\n> OTOH, my Data/ subtree currently ha...",0,0.0,0.005411
4,"On Wed, 2002-09-18 at 17:50, Matthias Saou wro...",0,0.0,0.000053


In [100]:
most_wrong = val_df[val_df["target"] != val_df["pred"]].sort_values("pred_prob", ascending=False)
most_wrong[:10]

,text,target,pred,pred_prob
567,http://news.bbc.co.uk/1/hi/world/asia-pacific/...,0,1.0,0.990566
140,Adjournment of Michel Fournier's big Jump in M...,0,1.0,0.989483
159,"Hey, it's not easy being green.\n\n\n\nleslie\...",0,1.0,0.970788
208,"If you're not in Doolin, beg, borrow, or steal...",0,1.0,0.938445
82,> Mmmph. You may actually need -L/usr/lib/X11 ...,0,1.0,0.908378
17,This is a multi-part message in MIME format.\n...,0,1.0,0.895626
96,Gary Lawrence Murphy:\n\n>This is for the whit...,0,1.0,0.889911
105,\n\nForwarded-by: Nev Dull <nev@sleepycat.com>...,0,1.0,0.841978
538,\n\nForwarded-by: Nev Dull <nev@sleepycat.com>...,0,1.0,0.841978
33,XDegrees was at the WebDAV Interoperability Te...,0,1.0,0.837781


0 = Ham
1 = Spam

In [108]:
for row in most_wrong[:10].itertuples(): # loop through the top 10 rows (change the index to view different rows)
  _, text, target, pred, prob = row
  print(f"Target: {target}, Pred: {int(pred)}, Prob: {prob}")
  print(f"Text:\n{text}\n")
  print("----\n")

Target: 0, Pred: 1, Prob: 0.9905661344528198
Text:
http://news.bbc.co.uk/1/hi/world/asia-pacific/2220132.stm



Tuesday, 27 August, 2002, 21:35 GMT 22:35 UK

Cambodia temple ruins yield treasure



Workers clearing dense jungle near the ruins of an ancient pagoda in

northern Cambodia have unearthed 31 Buddha statues - 27 of them solid gold.

The statues - which are 10 centimetres (4 inches) tall - are in good

condition and believed to be hundreds of years old.

They were found on Saturday as workers were rebuilding the Po Pich temple

about 100 km (65miles) north of the capital, Phnom Penh.

The pagoda, in the Batay district of Kampong Thom province, was torn down

during the reign of the Khmer Rouge in the 1970s and the area became

overgrown.

Community care

Deputy police chief of Kampong Thom province, Hang Sithim, said the statues

- three of which were silver and one bronze - were buried in about one metre

(3.4ft) of earth and each weigh around 500 grams (1lb).

''I think that

In [115]:
for row in most_wrong[-7:].itertuples():
  _, text, target, pred, prob = row
  print(f"Target: {target}, Pred: {int(pred)}, Prob: {prob}")
  print(f"Text:\n{text}\n")
  print("----\n")

Target: 1, Pred: 0, Prob: 0.15385407209396362
Text:
If you are a time traveler or alien disguised as human and or have

the technology to travel physically through time I need your help!



My life has been severely tampered with and cursed!!

I have suffered tremendously and am now dying!



I need to be able to:



Travel back in time.



Rewind my life including my age back to 4.



I am in very great danger and need this immediately!



I am aware of two types of time travel one in physical form and 

the other

in energy form where a snapshot of your brain is taken using 

either the

dimensional warp or the carbon copy replica device and then sends 

your consciousness

back through time to part with your younger self. Please explain

how safe and what your method involves.



I have a time machine now, but it has limited abilitys and is 

useless

without

a vortex.



If you can provide information on how to create vortex generator 

or

where I can get some of the blue glowing